In [1]:
import matplotlib.pyplot as plt
import numpy as np
#import regression_code as reg
#import read_data as read
import matplotlib.patches as mpatches
from matplotlib.patches import ConnectionPatch
import math

In [2]:
################################################## 
######     Import Package & Helper Function  #####
################################################## 

def loadDataArr(fileName):
    numFeat = len(open(fileName).readline().split('\t')) - 1
    xArr = []; yArr = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr =[]
        curLine = line.strip().split('\t')
        for i in range(numFeat):
            lineArr.append(float(curLine[i]))
        xArr.append(lineArr)
        yArr.append(float(curLine[-1]))
    return xArr, yArr

def loadDataList(fileName):
    dataList = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))      # map data to float()
        dataList.append(fltLine)
    return dataList

############################################# 
######     Standard linear regression  #####
############################################# 
    
'''   old functions with array as input and output
def stdLinReg (xArr_train, yArr_train):                                  # calculate the optimal weights for regression
    xMat_train = np.mat(xArr_train); yMat_train = np.mat(yArr_train).T               # convert array to matrix
    xTx = xMat_train.T * xMat_train                            
    if np.linalg.det(xTx) == 0.0:                                        # test the inversability
        print("This matrix is singular, can not do inverse")
        return
    wsMat_stdLinReg = xTx.I * (xMat_train.T*yMat_train)
    return wsMat_stdLinReg
    
def pred_stdLinReg(xArr_test, wsMat_stdLinReg):
    xMat = np.mat(xArr_test);                                  # convert array to matrix
    xCopy = xMat.copy(); xCopy.sort(0)                         # sort the points in ascending order for pyplot
    yMat_predOrdered = xCopy * wsMat_stdLinReg
    yMat_predNormal = xMat * wsMat_stdLinReg
    return yMat_predNormal, yMat_predOrdered
    

def showStdLinReg(xArr_test, yArr_test, yMat_predOrdered=None):
    xMat = np.mat(xArr_test); yMat = np.mat(yArr_test)         # convert array to matrix

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)                                  # create subplot
    if (np.shape(xMat)[1] == 1):                               # if there is 1 feature column
        ax.scatter(xMat[:,0].flatten().A[0], yMat.T[:,0].flatten().A[0], s=20, c='blue', alpha=.5)     # plot original data
    elif(np.shape(xMat)[1] == 2):                              # if there are 2 feature columns
        ax.scatter(xMat[:,1].flatten().A[0], yMat.T[:,0].flatten().A[0], s=20, c='blue', alpha=.5)     # plot original data
    
    if (np.all(yMat_predOrdered != None)):
        xCopy = xMat.copy(); xCopy.sort(0)                         # sort the points in ascending order for pyplot  
        if (np.shape(xMat)[1] == 1):
            ax.plot(xCopy[:,0], yMat_predOrdered, c='red')             # show regression line
        elif(np.shape(xMat)[1] == 2): 
            ax.plot(xCopy[:,1], yMat_predOrdered, c='red')             # show regression line
        plt.title('Linear Regression')                             # draw title    
    
    plt.xlabel('X')
    plt.show()
'''

def stdLinReg (dataMat):                    # format the dataset into the target variable Y and the independent variable X
    m,n = np.shape(dataMat)
    if n == 2:    # only 1D-x and y:  there are only one feature for x, namely no offset column, then add one for the linear regression
        X = np.mat(np.ones((m,n)))          # generate ones-matrix
        X[:,1:n] = dataMat[:,0:n-1]       # copy feature matrix to X, the 0th-column of matrix X is constant 1 as offset
    elif n>2:   # if there are more than one feature for X
        X = np.mat(np.ones((m,n-1)))        # generate ones-matrix
        X[:,0:n-1] = dataMat[:,0:n-1]       # copy feature matrix to X
    else:
        raise NameError('no valid input matrix')
    Y = np.mat(np.ones((m,1)))        
    Y = dataMat[:,-1]                  # copy target matrix to Y
    xTx = X.T*X
    if np.linalg.det(xTx) == 0.0:                   # check the inversability
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of Stop Condition')
    ws = xTx.I * (X.T * Y)                          # calculate the optimal weight matrix ws with least-squares method
    return ws,X,Y


def pred_stdLinReg(dataMat_test, wsMat_stdLinReg):
    m,n = np.shape(dataMat_test)
    if n == 2:    # only 1D-x and y:  there are only one feature for x, namely no offset column, then add one for the linear regression
        X = np.mat(np.ones((m,n)))       # generate ones-matrix
        X[:,1:n] = dataMat_test[:,0:n-1]       # copy feature matrix to X, the 0th-column of matrix X is constant 1 as offset
    elif n>2:   # if there are more than one feature for X
        X = np.mat(np.ones((m,n-1)))      # generate ones-matrix
        X[:,0:n-1] = dataMat_test[:,0:n-1]      # copy feature matrix to X
    else:
        raise NameError('no valid input matrix')
    
    yMat_pred = X * wsMat_stdLinReg
    return yMat_pred


def showStdLinReg(dataList, yMat_pred=None):
    n = len(dataList)                                                    
    xcord = []; ycord = []      

    if (np.shape(dataList)[1] == 2):            # check the number of columns
        for i in range(n):   
            xcord.append(dataList[i][0]); ycord.append(dataList[i][1])     # no offset in datalist, so use the first column as x
    elif(np.shape(dataList)[1] == 3):           
        for i in range(n):   
            xcord.append(dataList[i][1]); ycord.append(dataList[i][2])     # the first conlumn is offset, so use the second column as x

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)                                            
    ax.scatter(xcord, ycord, s = 20, c = 'blue',alpha = .5)      # plot the orignial data set
    if(np.all(yMat_pred != None)):
        yArr_pred = yMat_pred[:,0].flatten().A[0]                # convert matrix to array
        xMat_test = np.mat(xcord).T          
        srtInd = xMat_test.argsort(0)    
        xSort  = xMat_test[srtInd][:,0,:]                       # copy the xMat_test in ascending order for pyplot
        ax.plot(xSort[:], yArr_pred[srtInd], c = 'red')         # plot the prediction
        plt.title('Linear Regression')                             # draw title    
    plt.xlabel('X')
    plt.show()
    
    
def calError(yArr_actual, yArr_pred):                          # calculate the squared error
    return ((yArr_actual - yArr_pred) **2).sum() 
    
    
    
##################################################### 
######     Locally weighted linear regression  #####   
##################################################### 

'''
def localWeightLinReg(testPoint, xArr_train, yArr_train, k = 1.0):       # run lwlr-prediction  on a SINGLE value
    xMat_train = np.mat(xArr_train); yMat_train = np.mat(yArr_train).T             # convert array to matrix
    m = np.shape(xMat_train)[0]
    weights = np.mat(np.eye((m)))                          # Create diagonal matrix of weights
    for j in range(m):                                     # Populate weights with exponentially decaying values
        diffMat = testPoint - xMat_train[j, :]                                 
        weights[j, j] = np.exp(diffMat * diffMat.T/(-2.0 * k**2))     # locally weighted
    xTx = xMat_train.T * (weights * xMat_train)                                        
    if np.linalg.det(xTx) == 0.0:                                     # test the inversability
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I * (xMat_train.T * (weights * yMat_train))               # calculate the weights for "Locally weighted linear regression"
    return testPoint * ws                                              # return the predicted single value

def lwlr_Test(xArr_test, xArr_train, yArr_train, k=1.0):     # run lwlr-prediction  on a test ARRAY
    m = np.shape(xArr_test)[0]                               # check the size of data set
    yArr_pred = np.zeros(m)    
    for i in range(m):                                       # predict for each single element of test array
        yArr_pred[i] = localWeightLinReg(xArr_test[i], xArr_train, yArr_train, k)
    return yArr_pred
    
def showLwlr(xArr_test, yArr_test, yArr_pred=None, k=1.0):
    xMat_test = np.mat(xArr_test); yMat_test = np.mat(yArr_test)             # convert array to matrix
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)                                 # create subplot    
    if (np.shape(xMat_test)[1] == 1):
        ax.scatter(xMat_test[:,0].flatten().A[0], yMat_test.T[:,0].flatten().A[0], s=20, c='blue', alpha=.5)     # plot original data
    elif(np.shape(xMat_test)[1] == 2):  
        ax.scatter(xMat_test[:,1].flatten().A[0], yMat_test.T[:,0].flatten().A[0], s=20, c='blue', alpha=.5)     # plot original data
    
    if(np.all(yArr_pred != None)):
        if (np.shape(xMat_test)[1] == 1):
            srtInd = xMat_test[:,0].argsort(0)                        # return the ordered index according to the value of xMat[:,1]
            xSort  = xMat_test[srtInd][:,0,:]                         # copy the xMat in ascending order for pyplot
            ax.plot(xSort[:, 0], yArr_pred[srtInd], c = 'red')                  # plot the regression line
        elif(np.shape(xMat_test)[1] == 2): 
            srtInd = xMat_test[:,1].argsort(0)                        # return the ordered index according to the value of xMat[:,1]
            xSort  = xMat_test[srtInd][:,0,:]                         # copy the xMat in ascending order for pyplot
            ax.plot(xSort[:, 1], yArr_pred[srtInd], c = 'red')                  # plot the regression line
            plt.title('Locally weighted linear regression, k={}'.format(k))     # draw title    

    plt.xlabel('X')
    plt.show()
'''
def localWeightLinReg(testPoint, dataMat_train, k = 1.0):       # run lwlr-prediction  on a SINGLE value
    m,n = np.shape(dataMat_train)
    if n == 2:    # only 1D-x and y:  there are only one feature for x, namely no offset column, then add one for the linear regression
        XMat_train = np.mat(np.ones((m,n)))          # generate ones-matrix
        XMat_train[:,1:n] = dataMat_train[:,0:n-1]       # copy feature matrix to X, the 0th-column of matrix X is constant 1 as offset
    elif n>2:   # if there are more than one feature for X
        XMat_train = np.mat(np.ones((m,n-1)))        # generate ones-matrix
        XMat_train[:,0:n-1] = dataMat_train[:,0:n-1]       # copy feature matrix to X
    else:
        raise NameError('no valid input matrix')
    YMat_train = np.mat(np.ones((m,1)))        
    YMat_train = dataMat_train[:,-1]                  # copy target matrix to Y
    
    weights = np.mat(np.eye((m)))                          # Create diagonal matrix of weights
    for j in range(m):                                     # Populate weights with exponentially decaying values
        diffMat = testPoint - XMat_train[j,:]                                 
        weights[j, j] = np.exp(diffMat * diffMat.T/(-2.0 * k**2))     # locally weighted     
    xTx = XMat_train.T * (weights * XMat_train)                                        
    if np.linalg.det(xTx) == 0.0:                                     # test the inversability
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I * (XMat_train.T * (weights * YMat_train))               # calculate the weights for "Locally weighted linear regression"
    return testPoint * ws   
    

    
def lwlr_Test(dataMat_test, dataMat_train, k=1.0):     # run lwlr-prediction  on a test ARRAY
    m,n = np.shape(dataMat_test)
    if n == 2:                                      # only 1D-x and y:  there are only one feature for x, namely no offset column, then add one for the linear regression
        Xmat_test = np.mat(np.ones((m,n)))          # generate ones-matrix
        Xmat_test[:,1:n] = dataMat_test[:,0:n-1]    # copy feature matrix to X, the 0th-column of matrix X is constant 1 as offset
    elif n>2:                                       # if there are more than one feature for X
        Xmat_test = np.mat(np.ones((m,n-1)))        # generate ones-matrix
        Xmat_test[:,0:n-1] = dataMat_test[:,0:n-1]       # copy feature matrix to X
    else:
        raise NameError('no valid input matrix')
    
    yArr_pred = np.zeros(m)    
    for i in range(m):                                       # predict for each single element of test array
        yArr_pred[i] = localWeightLinReg(Xmat_test[i], dataMat_train, k)
    return yArr_pred


def showLwlr(dataList, yArr_pred=None, k=1.0):
    n = len(dataList)                                                    
    xcord = []; ycord = []      

    if (np.shape(dataList)[1] == 2):            # check the number of columns
        for i in range(n):   
            xcord.append(dataList[i][0]); ycord.append(dataList[i][1])     # no offset in datalist, so use the first column as x
    elif(np.shape(dataList)[1] == 3):           
        for i in range(n):   
            xcord.append(dataList[i][1]); ycord.append(dataList[i][2])     # the first conlumn is offset, so use the second column as x

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)                                 # create subplot   
    ax.scatter(xcord, ycord, s = 20, c = 'blue',alpha = .5)      # plot the orignial data set
    if(np.all(yArr_pred != None)):
        xMat_test = np.mat(xcord).T          
        srtInd = xMat_test.argsort(0)    
        xSort  = xMat_test[srtInd][:,0,:]                       # copy the xMat_test in ascending order for pyplot
        ax.plot(xSort[:], yArr_pred[srtInd], c = 'red')         # plot the prediction
        plt.title('Locally weighted linear regression, k={}'.format(k))     # draw title    
    plt.xlabel('X')
    plt.show()
    

    
    
    
########################################################### 
###############    Tree-based Methods    ###############
###########################################################



def showTree(dataList, yArr_pred=None, mode="regTree"):
    n = len(dataList)                                                    
    xcord = []; ycord = []      

    if (np.shape(dataList)[1] == 2):            # check the number of columns
        for i in range(n):   
            xcord.append(dataList[i][0]); ycord.append(dataList[i][1])    
    elif(np.shape(dataList)[1] == 3):
        for i in range(n):   
            xcord.append(dataList[i][1]); ycord.append(dataList[i][2])    

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)                                            
    ax.scatter(xcord, ycord, s = 20, c = 'blue',alpha = .5)      # plot the orignial data set
    if(np.all(yArr_pred != None)):
        xMat_test = np.mat(xcord).T          
        srtInd = xMat_test.argsort(0)    
        xSort  = xMat_test[srtInd][:,0,:]                       # copy the xMat_test in ascending order for pyplot
        ax.plot(xSort[:], yArr_pred[srtInd], c = 'red')         # plot the prediction
    if(mode == "regTree"):
        plt.title('Regression  Tree')         
    elif(mode == "modTree"):
        plt.title('Model  Tree')  
    plt.xlabel('X')
    plt.show()


##########################   functions for building Regression Tree    ##########################

def regLeaf(dataMat):                      # calculate the MEAN value as the model for a Leaf node
    return np.mean(dataMat[:,-1])          # dataMat[:,-1]: the last column of dataMat is Y

def regErr(dataMat):                       # calculate the TOTAL Squared Error of the target variables in a given dataset
    return np.var(dataMat[:,-1]) * np.shape(dataMat)[0]            # var(x): mean((x_i - x.mean())**2)
    # the smaller the variance is, the better the split. Goal: try to use LEAST split to seperate the whole data set

##########################   functions for building Model Tree    ##############################

def linearSolve(dataMat):  # format the dataset into the target variable Y and the independent variable X
    m,n = np.shape(dataMat)
    X = np.mat(np.ones((m,n))); Y = np.mat(np.ones((m,1)))      # generate ones-matrix
    X[:,1:n] = dataMat[:,0:n-1];       # copy feature matrix to X, the 0th-column of matrix X is constant 1 as offset
    Y = dataMat[:,-1]                  # copy target matrix to Y
    xTx = X.T*X
    if np.linalg.det(xTx) == 0.0:                   # check the inversability
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of Stop Condition')
    ws = xTx.I * (X.T * Y)                          # calculate the optimal weight matrix ws with least-squares method
    return ws,X,Y


def modelLeaf(dataMat):                             # generate a model for a leaf node
    ws,X,Y = linearSolve(dataMat)
    return ws

def modelErr(dataMat):                              # calculate the total squared error 
    ws,X,Y = linearSolve(dataMat)                   # of model against target
    yHat = X * ws
    return sum(np.power(Y - yHat, 2))        


##########################   functions for split    ##############################

def binSplitDataSet(dataMat, feature, value):                          # binary split
    mat0 = dataMat[np.nonzero(dataMat[:,feature] <= value)[0],:]       # np.nonzero(dataMat[:,feature] <= value)[0]: return index of target rows
    mat1 = dataMat[np.nonzero(dataMat[:,feature] > value)[0],:]        # np.nonzero(dataMat[:,feature] > value)[0]: return index of target rows
    return mat0, mat1

def chooseBestSplit(dataMat, leafType = regLeaf, errType = regErr, stopCond = (1,4)):
    minErrReduction = stopCond[0];         # stop condition: minimal Error reduction should be made through a new split
    minInstance = stopCond[1]              # stop condition: minimal amount of instances should be included in a leaf node
    if len(set(dataMat[:,-1].T.tolist()[0])) == 1:     # If all y-values are equal, NO SPLIT: Leaf node
        return None, leafType(dataMat)                 # calculate value for leaf node  
    
    m, n = np.shape(dataMat)      # get the size of dataset
    preError = errType(dataMat)   # setting the last feature as the best split and estimate its error for further compare
    bestError = float('inf');     # initialize bestError as an infinite value
    bestIndex = 0;                # initialize best splitting feature(Index) 
    bestValue = 0                 # initialize best splitting value
  
    for featIndex in range(n - 1):   # iterate all feature columns to find the splitting feature and splitting value
        for splitVal in set(dataMat[:,featIndex].T.tolist()[0]):    # iterate all x-values of ONE certain feature
            mat0, mat1 = binSplitDataSet(dataMat, featIndex, splitVal) 
            if (np.shape(mat0)[0] < minInstance) or (np.shape(mat1)[0] < minInstance): continue  # stop conditions met, NO SPLIT: Leaf node
            newError = errType(mat0) + errType(mat1)      # calculate the new error from two split sets
            if newError < bestError:                      # update if new error is smaller than best error
                bestIndex = featIndex
                bestValue = splitVal
                bestError = newError
                
    if (preError - bestError) < minErrReduction:                 # If stop conditions met, NO SPLIT: leaf node
        return None, leafType(dataMat)                           # calculate value for leaf node
    
    mat0, mat1 = binSplitDataSet(dataMat, bestIndex, bestValue)   # otherweise make the best split
    if (np.shape(mat0)[0] < minInstance) or (np.shape(mat1)[0] < minInstance):  # If stop conditions met, NO SPLIT: leaf node
        return None, leafType(dataMat)                            # calculate value for leaf node  
    return bestIndex, bestValue


##########################   functions for Creating and Pruning tree    ##############################

def createTree(dataMat_train, leafType = regLeaf, errType = regErr, stopCond = (1, 4)):
    feat, val = chooseBestSplit(dataMat_train, leafType, errType, stopCond)
    if feat == None: return val        # If stop condition met, return leaf value for the leaf node 
    retTree = {}                       # define retTree as dictionary
    retTree['spFeatIndex'] = feat
    retTree['spValue'] = val
    left_Set, right_Set = binSplitDataSet(dataMat_train, feat, val)
    retTree['left'] = createTree(left_Set, leafType, errType, stopCond)
    retTree['right'] = createTree(right_Set, leafType, errType, stopCond)
    return retTree  

def isTree(obj):      # check whether it is a tree or a leaf node
    return (type(obj).__name__ == 'dict') 
 

def getMean(tree):    # descend a tree untill it hits only leaf nodes, then take the MEAN value of both
    if isTree(tree['right']): 
        tree['right'] = getMean(tree['right'])
    if isTree(tree['left']): 
        tree['left'] = getMean(tree['left'])
    return (tree['left'] + tree['right']) / 2.0    


def prune(tree, testData):            # Post-pruning
    if np.shape(testData)[0] == 0:    # If no test data return MEAN value of left and right nodes 
        return getMean(tree)  
    
    if (isTree(tree['right']) or isTree(tree['left'])):  # split test data according to the trained tree
        lSet, rSet = binSplitDataSet(testData, tree['spFeatIndex'], tree['spValue'])
    if isTree(tree['left']): 
        tree['left'] = prune(tree['left'], lSet)      # prune the left subtree
    if isTree(tree['right']): 
        tree['right'] = prune(tree['right'], rSet)    # prune the right subtree
    if not isTree(tree['left']) and not isTree(tree['right']):     # if the leaf node of trained tree is reached
        
        #  test the total squared error with the value of leaf node of trained tree
        lSet, rSet = binSplitDataSet(testData, tree['spFeatIndex'], tree['spValue']) 
        errorNoMerge = np.sum(np.power(lSet[:,-1] - tree['left'], 2)) + np.sum(np.power(rSet[:,-1] - tree['right'], 2))

        #  test the total squared error with the MEAN value of leaf node of trained tree
        treeMean = (tree['left'] + tree['right']) / 2.0
        errorMerge = np.sum(np.power(testData[:,-1] - treeMean, 2))
        
        if errorMerge < errorNoMerge: 
  #          print("merging, tree['spFeatIndex']: {}, tree['spValue']:{}, tree['left']:{}, tree['right']:{}".format(tree['spFeatIndex'],tree['spValue'],tree['left'],tree['right']))
  #          print("treeMean:", treeMean)
  #          print("\n")
            return treeMean             # MERGE the left and right leaf node into one leaf node with MEAN value
        else: 
            return tree
    else: 
        return tree
    
    
    
##########################   functions for Prediction with Tree Model    ##############################

def regTreeEval(model, inDat):     # evaluate a Regression Tree leaf node
    return float(model)            # return the value at the leaf node


def modelTreeEval(model, inDat):   # evaluate a Model Tree leaf node
    n = np.shape(inDat)[1]
    X = np.mat(np.ones((1, n+1)))  # n+1 features, including the offset
    X[:, 1: n+1] = inDat           # copy inDat to X second to (n+1)th. column, X first column is offset with value '1'
    return float(X * model)        # return the forecasted value


# give one forecast for one data point, for a given tree.
def treeForecast(tree_trained, dataMat_test, modelEval=regTreeEval):

    if not isTree(tree_trained):                                # when a leaf node is hit, run modelEval()
        return modelEval(tree_trained, dataMat_test)
    
    if dataMat_test[:,tree_trained['spFeatIndex']] <= tree_trained['spValue']:    # follow the tree based on the input data 
        if isTree(tree_trained['left']):                                          # until a leaf node is hit 
            return treeForecast(tree_trained['left'], dataMat_test, modelEval)
        else:
            return modelEval(tree_trained['left'], dataMat_test)
    else:
        if isTree(tree_trained['right']):
            return treeForecast(tree_trained['right'], dataMat_test, modelEval)
        else:
            return modelEval(tree_trained['right'], dataMat_test)
        
'''
def createForeCast(tree_trained, dataMat_test, modelEval=regTreeEval):
    m = len(dataMat_test)
    yArr_pred = np.zeros(m)
    for i in range(m):                        #  run prediction for each SINGLE value of test set
        yArr_pred[i] = treeForecast(tree_trained, np.mat(dataMat_test[i]), modelEval)
    return yArr_pred
'''
def createForeCast(tree_trained, dataMat_test, modelEval=regTreeEval, iterate='false', treeX_trained=None, treeY_trained=None, iterNum=10, minDist=10):
    if (iterate == 'false'):                      #  just run forecast based on the test data
        m = len(dataMat_test)
        yArr_pred = np.zeros(m)
        for i in range(m):                        #  run prediction for each SINGLE value of test set
            yArr_pred[i] = treeForecast(tree_trained, np.mat(dataMat_test[i]), modelEval)
        return yArr_pred
    
    else:                                         #  iterate till the stop conditions met
        dataMat_pred = np.mat(np.zeros((iterNum,6)))
        dataMat_pred[0,0:4] = dataMat_test        # assign the position and speed of test data
        for i in range(iterNum-1):
            # calculate the acceleratex x(i), y(i)
            dataMat_pred[i,4] = treeForecast(treeX_trained, dataMat_pred[i,0:4], modelEval)
            dataMat_pred[i,5] = treeForecast(treeY_trained, dataMat_pred[i,0:4], modelEval)
            
            # calculate the NEW position x(i+1), y(i+1)
            dataMat_pred[i+1,0] = dataMat_pred[i,0] + dataMat_pred[i,2] + dataMat_pred[i,4]
            dataMat_pred[i+1,1] = dataMat_pred[i,1] + dataMat_pred[i,3] + dataMat_pred[i,5]
            
            # calculate the NEW speed x(i+1), y(i+1)
            dataMat_pred[i+1,2] = dataMat_pred[i,2] + dataMat_pred[i,4]
            dataMat_pred[i+1,3] = dataMat_pred[i,3] + dataMat_pred[i,5]
        
        dataMat_pred[-1,4] = treeForecast(treeX_trained, dataMat_pred[-1,0:4], modelEval)
        dataMat_pred[-1,5] = treeForecast(treeY_trained, dataMat_pred[-1,0:4], modelEval)
        
        return dataMat_pred



##########################   functions for plotting the tree    ##############################


def getNumLeafs(tree, numLeafNode=0):
   
    if isTree(tree['left']):       # check the 'left' part, whether it is a leaf node already
        numLeafNode = getNumLeafs(tree['left'], numLeafNode)
    else:
        numLeafNode += 1           # 'left' is a leaf node,then increment the total number of leaf node and then  check the 'right' of the SAME level!
    if isTree(tree['right']):      # check the 'right' of the SAME level
        numLeafNode = getNumLeafs(tree['right'], numLeafNode)
    else:
        return numLeafNode + 1     # if it is a lefe node, then return to the last stage
    
    return numLeafNode


def getDepth(tree, numTreeDepth=0, max =0):
    
    if not isTree(tree): 
        return 0
    if isTree(tree['left']):       # check the 'left' part, whether it is a tree 
        max = getDepth(tree['left'], numTreeDepth + 1, max)     # it is a tree, then go deep
    if isTree(tree['right']):      # check the 'right' of the SAME level
        max = getDepth(tree['right'], numTreeDepth + 1, max)
    else:
        numTreeDepth += 1
    max = numTreeDepth if numTreeDepth >= max else max
    return max             # return to the last stage
    
    
def getTreeDepth(tree):
    leftDepth = getDepth(tree['left'])
    rightDepth = getDepth(tree['right'])
    treeDepth = leftDepth if leftDepth >= rightDepth else rightDepth
    return treeDepth+1              # plus the very first splitt

###########################################################################################
###########################################################################################


def plotNode(nodeTxt, centerPt, parentPt, nodeType):                              # plot comment with arrow
    arrow_args = dict(arrowstyle="<-")                                            # set arrow format
#    font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)         # set chinese fond
    createPlot.ax1.annotate(nodeTxt, xy=parentPt,  xycoords='axes fraction',      # plot node
        xytext=centerPt, textcoords='axes fraction',
        va="center", ha="center", bbox=nodeType, arrowprops=arrow_args)
        

def plotMidText(cntrPt, parentPt, txtString):                                     #   plot transfer information bewteen tree and subtree
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cntrPt[0]                                # calculate position                  
    yMid = (parentPt[1]-cntrPt[1])/2.0 + cntrPt[1]
    createPlot.ax1.text(xMid, yMid, txtString, va="center", ha="center", rotation=30)
    
###########################################################################################
    
def plotTree(myTree, parentPt, nodeTxt, factorX=1, factorY=1):
    decisionNode = dict(boxstyle="sawtooth", fc="0.8")      # set decision node format，frame and arrow
    leafNode = dict(boxstyle="round4", fc="0.8")            # set leaf node format
    numLeafs = getNumLeafs(myTree)                          # get current number of total leaf nodes
#    depth = getTreeDepth(myTree)                           # get depth of tree
    cntrPt = (plotTree.xOff + (1.0 + float(numLeafs))/2.0/plotTree.totalW, plotTree.yOff)      #  define center position

##### plot decision node by plotNode(nodeTxt, centerPt, parentPt, nodeType)
    spFeatIndex = myTree['spFeatIndex']  
    spValue =  round(myTree['spValue'],2)                     # get the splitting point
    plotMidText(cntrPt, parentPt, '')
    plotNode("Feat:"+ str(spFeatIndex)+"\n"+"Val: " + str(spValue), cntrPt,  parentPt, decisionNode) 
    
    
#####  check leaf node
    if isTree(myTree['left']):                                          # if the leaf node is a tree, then run plotTree function
        plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD  #*factorY     # update yOff for leaf node
        plotTree(myTree['left'], cntrPt, '', factorX, factorY)
    else:                                                               # if the leaf node is a leaf node, then plot the node
        plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW             # update xOff for leaf node  !!!!!!!!
        plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD  #*factorY     # update yOff for leaf node
        leftNode = round(myTree['left'],2)                              # calculate value for leaf node
        plotNode(str(leftNode), (plotTree.xOff, plotTree.yOff), cntrPt, leafNode)      # plot left node
        plotMidText((plotTree.xOff, plotTree.yOff), cntrPt, "<=")                      # add "<=" 
        
    if isTree(myTree['right']):                                         # if the leaf node is a tree, then run plotTree function
        plotTree(myTree['right'], cntrPt, '', factorX, factorY)
    else:                                                               # if the leaf node is a leaf node, then plot the node
        plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW             # update xOff for leaf node
        rightNode = round(myTree['right'],2)                            # calculate value for leaf node
        plotNode(str(rightNode), (plotTree.xOff, plotTree.yOff), cntrPt, leafNode)      # plot left node
        plotMidText((plotTree.xOff, plotTree.yOff), cntrPt, ">")                        # add "<=" 
    plotTree.yOff = plotTree.yOff + 1.0/plotTree.totalD #*factorY                         # go back to the last stage   
    
    
def createPlot(inTree, factorX=1, factorY=1):
    fig = plt.figure(1, facecolor='white')                                                  # create fig
    fig.clf()                                                                               # clear fig
    axprops = dict(xticks=[], yticks=[])
    createPlot.ax1 = plt.subplot(111, frameon=False, **axprops)                             # delete x、y轴
    sumLeafNodes = getNumLeafs(inTree)
    treeDepth = getTreeDepth(inTree)
    plotTree.totalW = float(getNumLeafs(inTree))/factorX                                    # get total numbe of leaf nodes
    plotTree.totalD = float(getTreeDepth(inTree))/factorY                                           # get depth of tree
    plotTree.xOff = -0.5/plotTree.totalW; plotTree.yOff = 1.0;                              # define offset for x, y
    parent = (plotTree.xOff + (1.0 + float(getNumLeafs(inTree)))/2.0/plotTree.totalW, plotTree.yOff) 
    print("sum of leaf nodes: {} \ttree depth: {} \tfactorX: {}\tfactorY: {}".format(sumLeafNodes, treeDepth,factorX, factorY))
    plotTree(inTree, parent, '', factorX, factorY)                                       # plot tree
    plt.show()      
    
    
##########################################################################################





In [3]:
def plotTrack(posArr_org, posArr_pred=None, fig_size = (10,10)):
    fig = plt.figure(1, figsize=fig_size)
#    axprops = dict(xticks=[], yticks=[])
#    ax = plt.subplot(111, frameon=False, **axprops)          # delete x- and y-axis
    ax = plt.subplot(111)                                     # keep x- and y-axis
    posBegEnd = dict(boxstyle="round4", fc="0.8")            # set begiing and ending format

    minPosX_org = np.amin(posArr_org[:,0]); maxPosX_org = np.amax(posArr_org[:,0]);  # calculate min and max value of x-axis
    minPosY_org = np.amin(posArr_org[:,1]); maxPosY_org = np.amax(posArr_org[:,1]);  # calculate min and max value of y-axis

    for i in range(np.shape(posArr_org)[0]-1):     # plot the original route
        ax.annotate("",
                    xy=posArr_org[i+1], xycoords='data',        # xy: goal
                    xytext=posArr_org[i], textcoords='data',    # xytest: parent
                    bbox=posBegEnd,
                    arrowprops=dict(arrowstyle='-|>',
                    connectionstyle="arc3"),
        ) 
    ax.scatter(posArr_org[0][0], posArr_org[0][1], s = 100, c = 'green',alpha = 0.8, marker='s')       # plot the beginning point
    ax.scatter(posArr_org[-1][0], posArr_org[-1][1], s = 100, c = 'green',alpha = 1, marker='o')      # plot the ending point
    
    
    
    if (np.all(posArr_pred != None)):       # plot the predicted route
        for i in range(np.shape(posArr_pred)[0]-1):     
            ax.annotate("",
                        xy=posArr_pred[i+1], xycoords='data',        # xy: goal
                        xytext=posArr_pred[i], textcoords='data',    # xytest: parent
                        bbox=posBegEnd,
                        arrowprops=dict(arrowstyle='-|>',
                        connectionstyle="arc3", color='b'),
            ) 
        ax.scatter(posArr_pred[0][0], posArr_pred[0][1], s = 100, c = 'red',alpha = 0.4, marker='s')
        ax.scatter(posArr_pred[-1][0], posArr_pred[-1][1], s = 100, c = 'red',alpha = 1, marker='o')      # plot the ending point
        minPosX_pred = np.amin(posArr_pred[:,0]); maxPosX_pred = np.amax(posArr_pred[:,0]);  # calculate min and max value of x-axis
        minPosY_pred = np.amin(posArr_pred[:,1]); maxPosY_pred = np.amax(posArr_pred[:,1]);  # calculate min and max value of y-axis
        minX = minPosX_org if minPosX_org <= minPosX_pred  else minPosX_pred
        maxX = maxPosX_org if maxPosX_org >  maxPosX_pred  else maxPosX_pred
        minY = minPosY_org if minPosY_org <= minPosY_pred  else minPosY_pred
        maxY = maxPosY_org if maxPosY_org >  maxPosY_pred  else maxPosY_pred
        plt.xlim(minX*0.5, maxX*1.5)    # set the limit of x-axis
        plt.ylim(minY*0.5, maxY*1.5)    # set the limit of y-axis
    
    else:
        plt.xlim(minPosX_org*0.5, maxPosX_org*1.5)    # set the limit of x-axis
        plt.ylim(minPosY_org*0.5, maxPosY_org*1.5)    # set the limit of y-axis
        
    ax.set_ylim(ax.get_ylim()[::-1])      # invert the y-axis
    plt.show()

In [5]:
import numpy as np
import h5py
import pandas as pd

def read_parameter(group, sub):

    sub_group = group.get(sub)   
    position = np.array(sub_group.get('Positions'))
    velocity = np.array(sub_group.get('Velocity'))
    accerlation = np.array(sub_group.get('Acceleration'))
    
    return accerlation, position, velocity

def read_csv(Filename):
    data = pd.read_csv(Filename)
    print(data)

'''
def read_hdf5_auto(Filename):
    file = h5py.File(Filename)
    main_key = (list(file)[0])                  # "sample_run"
    group = file.get(main_key)                  # main group
  
    print('Filename is: \n', Filename)
    print('main_key: \n', (list(file)[0]))
    print('list(file.get(main_key)): \n', list(file.get(main_key)))
    
    for sub in list(file.get(main_key)):        # ['1', '2',...]
        accerlation, position, velocity = read_parameter(group, sub)

        print('sub: ', sub)                     
        print('position: \n',position)        
        print('\n') 
        print('velocity: \n',velocity)        
        print('\n') 
        print('accerlation: \n',accerlation)
        print('\n')    

    return accerlation, position, velocity
'''

E:\Anaconda\envs\trajectory\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'\ndef read_hdf5_auto(Filename):\n    file = h5py.File(Filename)\n    main_key = (list(file)[0])                  # "sample_run"\n    group = file.get(main_key)                  # main group\n  \n    print(\'Filename is: \n\', Filename)\n    print(\'main_key: \n\', (list(file)[0]))\n    print(\'list(file.get(main_key)): \n\', list(file.get(main_key)))\n    \n    for sub in list(file.get(main_key)):        # [\'1\', \'2\',...]\n        accerlation, position, velocity = read_parameter(group, sub)\n\n        print(\'sub: \', sub)                     \n        print(\'position: \n\',position)        \n        print(\'\n\') \n        print(\'velocity: \n\',velocity)        \n        print(\'\n\') \n        print(\'accerlation: \n\',accerlation)\n        print(\'\n\')    \n\n    return accerlation, position, velocity\n'

In [ ]:
def array_to_matrix()

### Read File and prepare the data ready for Training

In [140]:
path = 'samples/'
Filename = 'lin_zero_read_test.hdf5'   #  cub_400.hdf5
Filename = 'mid_PR20_PO20_F1_R10_mix.hdf5'   #  cub_400.hdf5
#Filename = 'mid_PR0_PO0_F1_R1.hdf5'
file = h5py.File(path + Filename)
#main_key = (list(file)[0])                  # "sample_run"
#group = file.get(main_key)                  # main group

In [144]:
key = list(file)
key

['mid_PR20_PO20_F1_R10']

In [149]:
group = file.get('mid_PR20_PO20_F1_R10')
group

<HDF5 group "/mid_PR20_PO20_F1_R10" (70 members)>

In [154]:
list(group)

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '8',
 '9']

In [156]:
read_parameter(group,'1')

(array([[-134.,  126.],
        [  82., -111.],
        [ -29.,  126.],
        [  19.,  -60.],
        [ -89.,   63.],
        [ 122., -111.],
        [ -45.,  156.],
        [  75., -108.],
        [ -16.,  -20.],
        [ -82.,   35.],
        [ -41.,   18.],
        [ 165., -106.],
        [-123.,  123.],
        [ -29., -108.],
        [  87.,  103.]]), array([[ 192.,  248.],
        [ 309.,  208.],
        [ 292.,  294.],
        [ 357.,  269.],
        [ 393.,  370.],
        [ 448.,  411.],
        [ 414.,  515.],
        [ 502.,  508.],
        [ 545.,  657.],
        [ 663.,  698.],
        [ 765.,  719.],
        [ 785.,  775.],
        [ 764.,  849.],
        [ 908.,  817.],
        [ 929.,  908.],
        [ 921.,  891.],
        [1000.,  977.]]), array([[117., -40.],
        [-17.,  86.],
        [ 65., -25.],
        [ 36., 101.],
        [ 55.,  41.],
        [-34., 104.],
        [ 88.,  -7.],
        [ 43., 149.],
        [118.,  41.],
        [102.,  21.],
        [ 

In [139]:
subGroup = file.get(list(file)[0])
subGroup

<HDF5 group "/lin_PR0_PO0_F1_R1" (1 members)>

<HDF5 group "/lin_PR0_PO0_F1_R1" (1 members)>

In [132]:
list(group)

['1']

In [121]:
len(group)

TypeError: object of type 'NoneType' has no len()

In [68]:
list(file)

['mid_PR20_PO20_F1_R10']

In [83]:
(list(file.get(main_key)))

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '8',
 '9']

In [108]:
sub_group = group.get('1')  
sub_group

<HDF5 group "/mid_PR20_PO20_F1_R10/1" (3 members)>

In [111]:
list(sub_group.get('Positions'))

[array([192., 248.]),
 array([309., 208.]),
 array([292., 294.]),
 array([357., 269.]),
 array([393., 370.]),
 array([448., 411.]),
 array([414., 515.]),
 array([502., 508.]),
 array([545., 657.]),
 array([663., 698.]),
 array([765., 719.]),
 array([785., 775.]),
 array([764., 849.]),
 array([908., 817.]),
 array([929., 908.]),
 array([921., 891.]),
 array([1000.,  977.])]

In [107]:
len(sub_group)

3

In [93]:
len(sub_group.get('Positions'))

17

In [106]:
a = sub_group.get('accerlation')
a

In [55]:
position = np.array(sub_group.get('Positions'))
position

array([[ 198.,  201.],
       [ 284.,  202.],
       [ 303.,  318.],
       [ 431.,  300.],
       [ 382.,  329.],
       [ 460.,  369.],
       [ 439.,  472.],
       [ 536.,  542.],
       [ 568.,  602.],
       [ 698.,  625.],
       [ 716.,  752.],
       [ 713.,  724.],
       [ 804.,  841.],
       [ 860.,  821.],
       [ 895.,  946.],
       [ 942.,  909.],
       [1001.,  988.]])

In [115]:
with h5py.File(path + Filename, 'r') as hdf:
    base_items = list(hdf.items())
    print('Items in the base directory: \n', base_items)
    print('\n')
    group = hdf.get('mid_PR20_PO20_F1_R10')
    group_items = list(group.items())
    print('Items in group: \n',group_items)
    print('\n')
    data_group = group.get('1')
    data_items = list(data_group.items())
    print('Items in data_group: \n',data_items)
    print('\n')

Items in the base directory: 
 [('lin_PR0_PO0_F1_R1', <HDF5 group "/lin_PR0_PO0_F1_R1" (1 members)>), ('zero_PR0_PO0_F1_R1', <HDF5 group "/zero_PR0_PO0_F1_R1" (1 members)>)]




AttributeError: 'NoneType' object has no attribute 'items'

In [101]:
a = []
a.__sizeof__()

40

In [ ]:
a.append('a')